In [1]:
pip install python-binance

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [1]:
from binance.client import Client
import pandas as pd

def obtener_datos(ticker, start_str, end_str=None, interval='1h'):
    # Inicializa el cliente de Binance. Reemplaza 'tu_api_key' y 'tu_api_secret' con tus claves de API.
    client = Client(api_key='2xASJJHFZW0H154RZrQmmEzxkgurAp5Udi9JQevAxazIVwwYveiT9RVq6eNPLYQi', api_secret='asGFYAIvvcf5GIDKtVRK5JP7MPp3x8hwAVNWQm1Jewe9qd7jmjA91Xw6p0l7FZvp')
    
    
    # Convierte las fechas de inicio y fin a milisegundos
    start_ts = int(pd.to_datetime(start_str).timestamp() * 1000)
    end_ts = None if end_str is None else int(pd.to_datetime(end_str).timestamp() * 1000)
    
    # Obtiene los klines (velas) de Binance
    klines = client.get_historical_klines(ticker, interval, start_str=start_ts, end_str=end_ts)
    
    # Convierte los klines a DataFrame
    df = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    
    # Convierte el tiempo de apertura de milisegundos a fecha legible
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    
    # Selecciona las columnas relevantes y convierte los valores a numérico
    df = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col])
    
    return df


# Colocar simbolo y fecha desde donde la quieres consultar 
df = obtener_datos('BTCUSDT', '2024-01-01')
print(df)

               open_time      open      high       low     close      volume
0    2024-01-01 00:00:00  42283.58  42554.57  42261.02  42475.23  1271.68108
1    2024-01-01 01:00:00  42475.23  42775.00  42431.65  42613.56  1196.37856
2    2024-01-01 02:00:00  42613.57  42638.41  42500.00  42581.10   685.21980
3    2024-01-01 03:00:00  42581.09  42586.64  42230.08  42330.49   794.80391
4    2024-01-01 04:00:00  42330.50  42399.99  42209.46  42399.99   715.41760
...                  ...       ...       ...       ...       ...         ...
4954 2024-07-25 10:00:00  64306.71  64314.01  63973.46  64165.68   834.35878
4955 2024-07-25 11:00:00  64165.68  64284.01  64089.72  64096.00  1115.62453
4956 2024-07-25 12:00:00  64096.01  64400.00  63900.24  64124.00  1628.67274
4957 2024-07-25 13:00:00  64123.99  64360.00  63680.00  64050.01  1635.35293
4958 2024-07-25 14:00:00  64050.01  64360.00  63456.70  64322.00  1958.05684

[4959 rows x 6 columns]


In [2]:
import pandas as pd
import pandas_ta

# Asegúrate de que tu DataFrame se llama 'df'
# Aplica Bollinger Bands y añade los resultados al DataFrame
df.ta.bbands(length=20, std=1.5, append=True)

# Imprime los nombres de las columnas para verificar los nombres correctos
print(df.columns)


Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'BBL_20_1.5',
       'BBM_20_1.5', 'BBU_20_1.5', 'BBB_20_1.5', 'BBP_20_1.5'],
      dtype='object')


In [26]:
# Asegúrate de que los nombres de las columnas en las siguientes líneas coincidan con los generados
# Por ejemplo, si los nombres correctos son 'BBL_20_1.5' y 'BBU_20_1.5', actualiza tu código así:
df['Upper Band'] = df['BBU_20_1.5']
df['Lower Band'] = df['BBL_20_1.5']

# Calcular la media móvil simple (SMA) de 20 periodos
df['SMA_20'] = df['close'].rolling(window=20).mean()

# Asegúrate de eliminar las filas con valores NaN que resulten de la operación de rolling
df.dropna(inplace=True)

# Esta función revisa las tendencias de las velas en relación a una media móvil y clasifica cada vela en categorías
def revisar_velas(df, backcanles, ma_column):
    # Inicializa una lista de categorías con ceros para los primeros 'backcanles' elementos
    categorias = [0 for _ in range(backcanles)]
    
    # Itera sobre el DataFrame desde 'backcanles' hasta el final del conjunto de datos
    for i in range(backcanles, len(df)):
        # Si todas las velas de cierre en el rango especificado están por encima de la media móvil, añade 'Alcista'
        if all(df['close'][i-backcanles:i] > df[ma_column][i-backcanles:i]):
            categorias.append('Alcista')
        # Si todas las velas de cierre en el rango especificado están por debajo de la media móvil, añade 'Bajista'
        elif all(df['close'][i-backcanles:i] < df[ma_column][i-backcanles:i]):
            categorias.append('Bajista')
        # Si ninguna de las condiciones anteriores se cumple, añade 'Neutro'
        else:
            categorias.append('Neutro')
    
    # Retorna la lista de categorías
    return categorias

# Aplica la función 'revisar_velas' al DataFrame 'datos' y almacena el resultado en una nueva columna llamada 'tendencia'
df['tendencia_velas'] = revisar_velas(df, 7, 'SMA_20')

# Verificar si hay alguna fila que cumpla con la condición de venta
sell_condition = (df['tendencia_velas'] == 'Bajista') & ((df['open'] > df['Upper Band']) & (df['close'] < df['Upper Band']))
print(df[sell_condition][['open', 'close', 'Lower Band', 'Upper Band', 'puntos_entrada']])

Empty DataFrame
Columns: [open, close, Lower Band, Upper Band, puntos_entrada]
Index: []


In [32]:
df

,open_time,open,high,low,close,volume,BBL_20_1.5,BBM_20_1.5,BBU_20_1.5,BBB_20_1.5,BBP_20_1.5,Upper Band,Lower Band,SMA_20,tendencia_velas,puntos_entrada
57,2024-01-03 09:00:00,45200.00,45500.00,45185.39,45344.67,2006.28796,44925.772388,45151.3890,45377.005612,0.999378,0.928339,45377.005612,44925.772388,45151.3890,0,neutro
58,2024-01-03 10:00:00,45344.66,45398.37,45045.95,45151.91,1918.59612,44923.466629,45146.0480,45368.629371,0.986050,0.513168,45368.629371,44923.466629,45146.0480,0,neutro
59,2024-01-03 11:00:00,45151.91,45174.07,43500.00,43702.88,7849.69163,44551.059852,45071.9510,45592.842148,2.311376,-0.814162,45592.842148,44551.059852,45071.9510,0,neutro
60,2024-01-03 12:00:00,43702.89,43716.00,40750.00,42789.38,20155.46959,44042.881401,44943.5280,45844.174599,4.007903,-0.695890,45844.174599,44042.881401,44943.5280,0,neutro
61,2024-01-03 13:00:00,42789.39,42820.92,42310.00,42529.40,5587.04560,43624.461010,44821.6905,46018.919990,5.342188,-0.457331,46018.919990,43624.461010,44821.6905,0,neutro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4954,2024-07-25 10:00:00,64306.71,64314.01,63973.46,64165.68,834.35878,63773.318594,65048.8450,66324.371406,3.921750,0.153804,66324.371406,63773.318594,65048.8450,Bajista,neutro
4955,2024-07-25 11:00:00,64165.68,64284.01,64089.72,64096.00,1115.62453,63695.107798,64944.8450,66194.582202,3.848611,0.160391,66194.582202,63695.107798,64944.8450,Bajista,neutro
4956,2024-07-25 12:00:00,64096.01,64400.00,63900.24,64124.00,1628.67274,63659.848801,64831.3450,66002.841199,3.613981,0.198102,66002.841199,63659.848801,64831.3450,Bajista,neutro
4957,2024-07-25 13:00:00,64123.99,64360.00,63680.00,64050.01,1635.35293,63675.693089,64704.5450,65733.396911,3.180153,0.181910,65733.396911,63675.693089,64704.5450,Bajista,neutro


In [40]:
# Imprimir los primeros valores de las columnas relevantes para depuración
print("Valores de las columnas relevantes:")
print(df[['tendencia_velas', 'open', 'close', 'Lower Band', 'Upper Band']].head(10))

# Verificar las condiciones de compra y venta
buy_condition = (df['tendencia_velas'] == 'Alcista') & ((df['open'] < df['Lower Band']) & (df['close'] > df['Lower Band']))
sell_condition = (df['tendencia_velas'] == 'Bajista') & ((df['open'] > df['Upper Band']) & (df['close'] < df['Upper Band']))

# Imprimir las condiciones para depuración
print("Condiciones de compra (buy_condition):")
print(df[buy_condition])

print("Condiciones de venta (sell_condition):")
print(df[sell_condition])

# Aplicar las condiciones
df.loc[buy_condition, 'puntos_entrada'] = 'Alcista'
df.loc[sell_condition, 'puntos_entrada'] = 'Bajista'

# Filtrar y mostrar las filas donde 'puntos_entrada' no es 'neutro'
result = df[df['puntos_entrada'] != 'neutro']
print("Filas con 'puntos_entrada' no 'neutro':")
print(result)

Valores de las columnas relevantes:
   tendencia_velas      open     close    Lower Band    Upper Band
57               0  45200.00  45344.67  44925.772388  45377.005612
58               0  45344.66  45151.91  44923.466629  45368.629371
59               0  45151.91  43702.88  44551.059852  45592.842148
60               0  43702.89  42789.38  44042.881401  45844.174599
61               0  42789.39  42529.40  43624.461010  46018.919990
62               0  42529.40  42179.99  43204.555291  46141.355709
63               0  42180.00  43000.01  43011.319035  46130.643965
64          Neutro  43000.01  42992.67  42841.279125  46121.122875
65          Neutro  42992.68  42906.00  42669.029554  46071.998446
66         Bajista  42906.00  42255.72  42413.674930  46054.752070
Condiciones de compra (buy_condition):
Empty DataFrame
Columns: [open_time, open, high, low, close, volume, BBL_20_1.5, BBM_20_1.5, BBU_20_1.5, BBB_20_1.5, BBP_20_1.5, Upper Band, Lower Band, SMA_20, tendencia_velas, puntos_ent

In [39]:

df[df['puntos_entrada'] != 'neutro']

,open_time,open,high,low,close,volume,BBL_20_1.5,BBM_20_1.5,BBU_20_1.5,BBB_20_1.5,BBP_20_1.5,Upper Band,Lower Band,SMA_20,tendencia_velas,puntos_entrada


In [42]:
# Imprimir los primeros valores de las columnas relevantes para depuración
print("Valores de las columnas relevantes:")
print(df[['tendencia_velas', 'open', 'close', 'Lower Band', 'Upper Band']].head(10))

# Verificar las condiciones de compra y venta
buy_condition = (df['tendencia_velas'] == 'Alcista') & ((df['open'] < df['Lower Band']) & (df['close'] > df['Lower Band']))
sell_condition = (df['tendencia_velas'] == 'Bajista') & ((df['open'] > df['Upper Band']) & (df['close'] < df['Upper Band']))

# Imprimir las condiciones para depuración
print("Condiciones de compra (buy_condition):")
print(df[buy_condition])

print("Condiciones de venta (sell_condition):")
print(df[sell_condition])

# Aplicar las condiciones
df.loc[buy_condition, 'puntos_entrada'] = 'Alcista'
df.loc[sell_condition, 'puntos_entrada'] = 'Bajista'

# Filtrar y mostrar las filas donde 'puntos_entrada' no es 'neutro'
result = df[df['puntos_entrada'] != 'neutro']
print("Filas con 'puntos_entrada' no 'neutro':")
print(result)

# Verificar los valores únicos de 'tendencia_velas'
print("Valores únicos de 'tendencia_velas':")
print(df['tendencia_velas'].unique())

Valores de las columnas relevantes:
   tendencia_velas      open     close    Lower Band    Upper Band
57               0  45200.00  45344.67  44925.772388  45377.005612
58               0  45344.66  45151.91  44923.466629  45368.629371
59               0  45151.91  43702.88  44551.059852  45592.842148
60               0  43702.89  42789.38  44042.881401  45844.174599
61               0  42789.39  42529.40  43624.461010  46018.919990
62               0  42529.40  42179.99  43204.555291  46141.355709
63               0  42180.00  43000.01  43011.319035  46130.643965
64          Neutro  43000.01  42992.67  42841.279125  46121.122875
65          Neutro  42992.68  42906.00  42669.029554  46071.998446
66         Bajista  42906.00  42255.72  42413.674930  46054.752070
Condiciones de compra (buy_condition):
Empty DataFrame
Columns: [open_time, open, high, low, close, volume, BBL_20_1.5, BBM_20_1.5, BBU_20_1.5, BBB_20_1.5, BBP_20_1.5, Upper Band, Lower Band, SMA_20, tendencia_velas, puntos_ent